# 特征工程

## 数据处理原则

+ 0代表的是值为0，不代表不存在
+ -1代表不存在，所以**fillna**的时候需要考虑清楚

## 特征

+ [特征工程方案](https://github.com/wepe/O2O-Coupon-Usage-Forecast)
+ [xgboost实现方式](https://github.com/Mryangkaitong/python-Machine-learning/tree/master/Xgboost)
+ [机器学习中如何利用id类特征？](https://www.zhihu.com/question/34819617)
+ [特征工程-特征处理-代码](http://www.zeroyx.com/index.php?r=site/art&id=17&title_id=98)
+ [Xgboost实践+第一名天池o2o优惠券的使用预测思路完整版](https://blog.csdn.net/weixin_42001089/article/details/85013073?spm=5176.9876270.0.0.191fe44aVvRL69)

用户

+ 用户领取商家的优惠券后核销率
+ 用户对每个商家的优惠券核销次数占用户总的核销次数的比重
+ 用户对每个商家的优惠券核销次数占商家总的核销次数的比重

优惠券

+ 优惠券信息 - 计算折扣率
+ 优惠券信息 - 优惠券类型
+ 优惠券信息 - 优惠券最低消费金额
+ 优惠券信息 - 每月收到券的时间
+ 优惠券信息 - 每周收到券的时间

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from datetime import datetime
from scipy import stats

In [2]:
offline_df = pd.read_csv('ccf_offline_stage1_train.csv')
offline_df = offline_df.fillna(0)
offline_df = offline_df[offline_df['Date_received'] < 20160501]
offline_df.shape

(1448571, 7)

In [3]:
offline_df.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date
count,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06
mean,3.690789e+06,4.123009e+03,3.656079e+03,2.324601e+00,1.039580e+07,1.036206e+07
std,2.123726e+06,2.393802e+03,4.634087e+03,3.504746e+00,1.007517e+07,1.007626e+07
min,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.845660e+06,2.099000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.697100e+06,3.532000e+03,1.110000e+02,0.000000e+00,2.016012e+07,2.016011e+07
75%,5.531531e+06,6.424000e+03,7.610000e+03,3.000000e+00,2.016020e+07,2.016042e+07
max,7.361032e+06,8.856000e+03,1.404500e+04,1.000000e+01,2.016043e+07,2.016063e+07


In [4]:
offline_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,0.0,0,0.0,0.0,20160217.0
2,1439408,2632,8591.0,20:1,0.0,20160217.0,0.0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,0.0
5,1439408,2632,0.0,0,0.0,0.0,20160516.0
7,1832624,3381,7610.0,200:20,0.0,20160429.0,0.0


In [5]:
online_df = pd.read_csv('ccf_online_stage1_train.csv')
online_df = online_df.fillna(0)
online_df.shape

(11429826, 7)

In [6]:
online_df.describe()

,User_id,Merchant_id,Action,Date_received,Date
count,1.142983e+07,1.142983e+07,1.142983e+07,1.142983e+07,1.142983e+07
mean,1.074683e+07,3.436686e+04,2.348193e-01,1.538697e+06,1.900349e+07
std,4.137712e+06,1.441243e+04,5.426314e-01,5.352861e+06,4.688829e+06
min,4.000000e+00,1.000100e+04,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.019827e+07,2.090100e+04,0.000000e+00,0.000000e+00,2.016022e+07
50%,1.196970e+07,3.420000e+04,0.000000e+00,0.000000e+00,2.016040e+07
75%,1.373506e+07,4.741500e+04,0.000000e+00,0.000000e+00,2.016052e+07
max,1.550000e+07,6.000000e+04,2.000000e+00,2.016062e+07,2.016063e+07


In [7]:
online_df.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513.0,0.0
1,13740231,34805,1,0,0,0.0,20160321.0
2,14336199,18907,0,0,0,0.0,20160618.0
3,14336199,18907,0,0,0,0.0,20160618.0
4,14336199,18907,0,0,0,0.0,20160618.0


In [8]:
df = offline_df.copy()

## 特征提取

### 基础处理

In [9]:
def cal_duration(row):
    if row['Coupon_id'] > 0 and row['Date_received'] > 0 and row['Date'] > 0:
        date_received = datetime.strptime(str(int(row['Date_received'])), '%Y%m%d')
        date_consumed = datetime.strptime(str(int(row['Date'])), '%Y%m%d')
        delta = date_consumed - date_received
        return delta.days + 1
    return 0

In [10]:
def cal_previous_duration(row):
    if row['User_id'] == row['Previous_user_id'] and row['Date_received'] > 0 and row['Previous_date_received'] > 0:
        date_received = datetime.strptime(str(int(row['Date_received'])), '%Y%m%d')
        date_previous_received = datetime.strptime(str(int(row['Previous_date_received'])), '%Y%m%d')
        delta = date_received - date_previous_received
        return delta.days + 1
    
    return 0

In [11]:
def cal_next_duration(row):
    if row['User_id'] == row['Next_user_id'] and row['Date_received'] > 0 and row['Next_date_received'] > 0:
        date_received = datetime.strptime(str(int(row['Date_received'])), '%Y%m%d')
        date_next_received = datetime.strptime(str(int(row['Next_date_received'])), '%Y%m%d')
        delta = date_next_received - date_received
        return delta.days + 1
    
    return 0

In [12]:
# 优惠券信息 - 计算折扣率
def cal_discount(row):
    if isinstance(row['Discount_rate'], int):
        return float(row['Discount_rate'])
    
    if row['Discount_rate'] == 'fixed':
        return 0.0
    
    arr = row['Discount_rate'].split(':')
    if len(arr) == 2:
        return (float(arr[0]) - float(arr[1])) / float(arr[0])
    else:
        return float(row['Discount_rate'])

def set_coupon_type(row):
    if isinstance(row['Discount_rate'], int):
        return 1
    
    if row['Discount_rate'] == 'fixed':
        return 2
    
    arr = row['Discount_rate'].split(':')
    if len(arr) == 2:
        return 1
    else:
        return 0

def base_consume(row):
    if isinstance(row['Discount_rate'], int):
        return float(row['Discount_rate'])
    
    if row['Discount_rate'] == 'fixed':
        return 0.0
    
    arr = row['Discount_rate'].split(':')
    if len(arr) == 2:
        return float(arr[0])
    else:
        return 0.0

def get_day_in_month_4_received_day(row):
    if isinstance(row['Date_received'], int) or int(row['Date_received']) <= 0:
        return 0.0
    
    date_received = datetime.strptime(str(int(row['Date_received'])), '%Y%m%d')
    return date_received.day

def get_day_in_week_4_received_day(row):
    if isinstance(row['Date_received'], int) or int(row['Date_received']) <= 0:
        return 0.0
    
    date_received = datetime.strptime(str(int(row['Date_received'])), '%Y%m%d')
    return (date_received.weekday() + 1)

In [13]:
def base_data_process(df):
    df = df.sort_values(by=['User_id', 'Date_received'], ascending=True)

    df['Previous_user_id'] = df['User_id'].shift(1)
    df['Previous_date_received'] = df['Date_received'].shift(1)

    df['Next_user_id'] = df['User_id'].shift(-1)
    df['Next_date_received'] = df['Date_received'].shift(-1)

    df.fillna(0)
    
    df['Distance'] = df['Distance'] + 1
    df['Duration'] = df.apply(lambda row: cal_duration(row), axis=1)
    df['Previous_duration'] = df.apply(lambda row: cal_previous_duration(row), axis=1)
    df['Next_duration'] = df.apply(lambda row: cal_next_duration(row), axis=1)
    
    df = df.drop(['Next_user_id', 'Previous_user_id'], axis=1)
    
    df['Base_consume'] = df.apply(lambda row: base_consume(row), axis=1)
    df['Day_in_month_received'] = df.apply(lambda row: get_day_in_month_4_received_day(row), axis=1)
    df['Day_in_week_received'] = df.apply(lambda row: get_day_in_week_4_received_day(row), axis=1)
    df['Discount'] = df.apply(lambda row: cal_discount(row), axis=1)
    df['Coupon_type'] = df.apply(lambda row: set_coupon_type(row), axis=1)
    
    df['Is_in_day_consume'] = df.apply(lambda row: 1 if row['Duration'] > 0 and row['Duration'] < 17 else 0, axis = 1)
    
    return df

In [14]:
df = base_data_process(df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448571 entries, 679792 to 1111815
Data columns (total 18 columns):
User_id                   1448571 non-null int64
Merchant_id               1448571 non-null int64
Coupon_id                 1448571 non-null float64
Discount_rate             1448571 non-null object
Distance                  1448571 non-null float64
Date_received             1448571 non-null float64
Date                      1448571 non-null float64
Previous_date_received    1448570 non-null float64
Next_date_received        1448570 non-null float64
Duration                  1448571 non-null int64
Previous_duration         1448571 non-null int64
Next_duration             1448571 non-null int64
Base_consume              1448571 non-null float64
Day_in_month_received     1448571 non-null float64
Day_in_week_received      1448571 non-null float64
Discount                  1448571 non-null float64
Coupon_type               1448571 non-null int64
Is_in_day_consume         14

In [16]:
df.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Previous_date_received,Next_date_received,Duration,Previous_duration,Next_duration,Base_consume,Day_in_month_received,Day_in_week_received,Discount,Coupon_type,Is_in_day_consume
679792,4,1433,8735.0,30:5,11.0,20160214.0,0.0,NaN,20160129.0,0,0,0,30.0,14.0,7.0,0.833333,1,0
678050,35,3381,9776.0,10:5,1.0,20160129.0,0.0,20160214.0,20160129.0,0,0,1,10.0,29.0,5.0,0.500000,1,0
678052,35,3381,11951.0,200:20,1.0,20160129.0,0.0,20160129.0,20160130.0,0,1,2,200.0,29.0,5.0,0.900000,1,0
678051,35,3381,1807.0,300:30,1.0,20160130.0,0.0,20160129.0,20160130.0,0,2,1,300.0,30.0,6.0,0.900000,1,0
678053,35,3381,11951.0,200:20,1.0,20160130.0,0.0,20160130.0,20160125.0,0,1,0,200.0,30.0,6.0,0.900000,1,0
1264846,36,1041,13490.0,30:5,5.0,20160125.0,0.0,20160130.0,20160125.0,0,0,1,30.0,25.0,1.0,0.833333,1,0
1264847,36,5717,12349.0,20:5,9.0,20160125.0,0.0,20160125.0,20160129.0,0,1,0,20.0,25.0,1.0,0.750000,1,0
97505,64,2146,11173.0,100:10,3.0,20160129.0,0.0,20160125.0,20160131.0,0,0,0,100.0,29.0,5.0,0.900000,1,0
1263168,110,7019,3887.0,100:10,3.0,20160131.0,0.0,20160129.0,20160131.0,0,0,1,100.0,31.0,7.0,0.900000,1,0
1263169,110,6454,14031.0,100:10,11.0,20160131.0,0.0,20160131.0,20160131.0,0,1,1,100.0,31.0,7.0,0.900000,1,0


### 特征提取的公用方法

In [17]:
def fetch(row, col_name, series):
    vid = row[col_name]
    if vid in series.index:
        return series[vid]
    else:
        return 0.0

def join(df, col, series, key):
    return multi_join(df, col, series, [key])

def multi_fetch(row, col_name_1, col_name_2, series):
    vid_1 = row[col_name_1]
    vid_2 = row[col_name_2]
    
    if (vid_1, vid_2) in series.index:
        return series[vid_1, vid_2]
    else:
        return 0.0

def multi_join(df, col, series, keys):
    t = series.to_frame()
    t.columns = [col]
    return pd.merge(df, t, on=keys, how='left')

### 数据分层

In [18]:
df['Coupon_id'] = df['Coupon_id'].astype('int64', copy=True)

# 领取优惠券的信息
received_df = df[df['Coupon_id'] > 0]

# 消费的信息
consume_df = df[df['Date'] > 0]

# 消费同时15天内使用优惠券的信息
used_coupon_df = df[df['Is_in_day_consume'] == 1]

### 用户特征的提取

在模型的训练中，不再使用User_id，因为User_id不带有任何信息，我们需要从各种信息中获取信息来刻画一个User_id

#### 线下

用户领取优惠券并核销信息

+ 用户领取优惠券次数
+ 用户线下门店消费次数
+ 用户获得优惠券并核销次数
+ 用户获得优惠券但没有消费的次数
+ 用户15天内线下门店消费并核销优惠券次数
+ 用户领取优惠券后进行核销率
+ 用户15天内线下门店领取优惠券后进行核销率
+ 用户核销优惠券的平均/最低/最高消费折率
+ 用户15天内线下门核销优惠券的平均/最低/最高消费折率
+ 用户核销过优惠券的不同商家数量，及其占所有不同商家的比重
+ 用户核销过的不同优惠券数量，及其占所有不同优惠券的比重
+ 用户平均核销每个商家多少张优惠券
+ 用户核销优惠券中的平均/最大/最小 用户-商家距离

以下不参与

+ 用户满050/50200/200~500 减的优惠券核销率
+ 用户核销满050/50200/200~500减的优惠券占所有核销优惠券的比重

In [19]:
# 获取所有在线下消费过的用户id
user_df = df['User_id'].drop_duplicates()
user_df = user_df.to_frame()

In [20]:
# 用户领取优惠券次数
user_receive_count = received_df.groupby(['User_id']).size()
# 用户线下门店消费次数
user_consume_count = consume_df.groupby(['User_id']).size()
# 用户15天内线下门店消费并核销优惠券次数
user_used_count = used_coupon_df.groupby(['User_id']).size()

# 用户特征
user_df = join(user_df, 'User_receive_count', user_receive_count, 'User_id')
user_df = join(user_df, 'User_consume_count', user_consume_count, 'User_id')
user_df = join(user_df, 'User_used_count', user_used_count, 'User_id')

user_df = user_df.fillna(0)
user_df['User_not_used_count'] = user_df.apply(lambda r: r['User_receive_count'] - r['User_used_count'], axis=1)

# 用户15天内线下门店领取优惠券后进行核销率
user_df['User_used_coupon_rate'] = user_df.apply(lambda r: r['User_used_count'] / r['User_receive_count'], axis=1)

user_df = user_df.fillna(0)
user_df = user_df.replace(math.inf, 0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [21]:
used_coupon_rate_max = user_df['User_used_coupon_rate'].max()
used_coupon_rate_min = user_df['User_used_coupon_rate'].min()
used_coupon_rate_mean = user_df['User_used_coupon_rate'].mean()

user_df['User_used_coupon_rate_max'] = used_coupon_rate_max
user_df['User_used_coupon_rate_min'] = used_coupon_rate_min
user_df['User_used_coupon_rate_mean'] = used_coupon_rate_mean

In [22]:
user_receive_coupon_merchant_count =  received_df[['User_id','Merchant_id']].drop_duplicates().groupby(['User_id']).size()
user_consume_merchant_count =  consume_df[['User_id','Merchant_id']].drop_duplicates().groupby(['User_id']).size()
user_used_coupon_merchant_count =  used_coupon_df[['User_id','Merchant_id']].drop_duplicates().groupby(['User_id']).size()

user_df = join(user_df, 'User_receive_coupon_merchant_count', user_receive_coupon_merchant_count, 'User_id')
user_df = join(user_df, 'User_consume_merchant_count', user_consume_merchant_count, 'User_id')
user_df = join(user_df, 'User_used_coupon_merchant_count', user_used_coupon_merchant_count, 'User_id')
user_df = user_df.fillna(0)

merchant_count = df['Merchant_id'].drop_duplicates().count()
user_df['User_used_coupon_merchant_occ'] = user_df['User_used_coupon_merchant_count'] / merchant_count

user_df = user_df.fillna(0)
user_df = user_df.replace(math.inf, 0)

In [23]:
user_receive_different_coupon_count =  received_df[['User_id','Coupon_id']].drop_duplicates().groupby(['User_id']).size()
user_used_different_coupon_count =  used_coupon_df[['User_id','Coupon_id']].drop_duplicates().groupby(['User_id']).size()

user_df = join(user_df, 'User_receive_different_coupon_count', user_receive_different_coupon_count, 'User_id')
user_df = join(user_df, 'User_used_different_coupon_count', user_used_different_coupon_count, 'User_id')
user_df = user_df.fillna(0)

coupon_count = df[df['Coupon_id']>0]['Coupon_id'].drop_duplicates().count()

user_df['User_receive_different_coupon_occ'] = user_df['User_receive_different_coupon_count'] / coupon_count
user_df['User_used_different_coupon_occ'] = user_df['User_used_different_coupon_count'] / coupon_count

# 用户平均核销每个商家多少张优惠券
user_df['User_receive_coupon_mean'] = user_df['User_receive_count'] / user_df['User_receive_coupon_merchant_count']
user_df['User_used_coupon_mean'] = user_df['User_used_count'] / user_df['User_receive_coupon_merchant_count']

user_df = user_df.fillna(0)
user_df = user_df.replace(math.inf, 0)

In [24]:
user_distance_used_mean = used_coupon_df[['User_id', 'Distance']].groupby(['User_id']).mean()
user_distance_used_max = used_coupon_df[['User_id', 'Distance']].groupby(['User_id']).max()
user_distance_used_min = used_coupon_df[['User_id', 'Distance']].groupby(['User_id']).min()

user_distance_df = pd.DataFrame({'User_id': user_distance_used_mean.index, 'User_distance_used_mean':user_distance_used_mean['Distance'], 'User_distance_used_max':user_distance_used_max['Distance'], 'User_distance_used_min':user_distance_used_min['Distance']})

user_df = pd.merge(user_df, user_distance_df, on=['User_id'], how='left')
user_df = user_df.fillna(0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: FutureWarning: 'User_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
user_duration_used_mean = used_coupon_df[['User_id', 'Duration']].groupby(['User_id']).mean()
user_duration_used_max = used_coupon_df[['User_id', 'Duration']].groupby(['User_id']).max()
user_duration_used_min = used_coupon_df[['User_id', 'Duration']].groupby(['User_id']).min()

user_duration_df = pd.DataFrame({'User_id': user_duration_used_mean.index, 'User_duration_used_mean':user_duration_used_mean['Duration'], 'User_duration_used_max':user_duration_used_max['Duration'], 'User_duration_used_min':user_duration_used_min['Duration']})

user_df = pd.merge(user_df, user_duration_df, on=['User_id'], how='left')
user_df = user_df.fillna(0)

In [26]:
user_previous_duration_used_mean = used_coupon_df[['User_id', 'Previous_duration']].groupby(['User_id']).mean()
user_previous_duration_used_max = used_coupon_df[['User_id', 'Previous_duration']].groupby(['User_id']).max()
user_previous_duration_used_min = used_coupon_df[['User_id', 'Previous_duration']].groupby(['User_id']).min()

user_previous_duration_df = pd.DataFrame({'User_id': user_previous_duration_used_mean.index, 'User_previous_duration_used_mean':user_previous_duration_used_mean['Previous_duration'], 'User_previous_duration_used_max':user_previous_duration_used_max['Previous_duration'], 'User_previous_duration_used_min':user_previous_duration_used_min['Previous_duration']})

user_df = pd.merge(user_df, user_previous_duration_df, on=['User_id'], how='left')
user_df = user_df.fillna(0)

In [27]:
user_next_duration_used_mean = used_coupon_df[['User_id', 'Next_duration']].groupby(['User_id']).mean()
user_next_duration_used_max = used_coupon_df[['User_id', 'Next_duration']].groupby(['User_id']).max()
user_next_duration_used_min = used_coupon_df[['User_id', 'Next_duration']].groupby(['User_id']).min()

user_next_duration_df = pd.DataFrame({'User_id': user_next_duration_used_mean.index, 'User_next_duration_used_mean':user_next_duration_used_mean['Next_duration'], 'User_next_duration_used_max':user_next_duration_used_max['Next_duration'], 'User_next_duration_used_min':user_next_duration_used_min['Next_duration']})

user_df = pd.merge(user_df, user_next_duration_df, on=['User_id'], how='left')
user_df = user_df.fillna(0)

In [28]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476840 entries, 0 to 476839
Data columns (total 31 columns):
User_id                                476840 non-null int64
User_receive_count                     476840 non-null float64
User_consume_count                     476840 non-null float64
User_used_count                        476840 non-null float64
User_not_used_count                    476840 non-null float64
User_used_coupon_rate                  476840 non-null float64
User_used_coupon_rate_max              476840 non-null float64
User_used_coupon_rate_min              476840 non-null float64
User_used_coupon_rate_mean             476840 non-null float64
User_receive_coupon_merchant_count     476840 non-null float64
User_consume_merchant_count            476840 non-null float64
User_used_coupon_merchant_count        476840 non-null float64
User_used_coupon_merchant_occ          476840 non-null float64
User_receive_different_coupon_count    476840 non-null float64
User_used

In [29]:
user_df.describe()

,User_id,User_receive_count,User_consume_count,User_used_count,User_not_used_count,User_used_coupon_rate,User_used_coupon_rate_max,User_used_coupon_rate_min,User_used_coupon_rate_mean,User_receive_coupon_merchant_count,...,User_distance_used_min,User_duration_used_mean,User_duration_used_max,User_duration_used_min,User_previous_duration_used_mean,User_previous_duration_used_max,User_previous_duration_used_min,User_next_duration_used_mean,User_next_duration_used_max,User_next_duration_used_min
count,4.768400e+05,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000,476840.0,476840.0,4.768400e+05,476840.000000,...,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000,476840.000000
mean,3.685771e+06,1.566498,1.561400,0.076135,1.490364,0.031743,1.0,0.0,3.174270e-02,1.329064,...,0.095965,0.305082,0.333621,0.279312,0.397497,0.507843,0.308248,0.268943,0.337285,0.216404
std,2.123869e+06,1.776406,4.056561,0.493920,1.667525,0.156366,0.0,0.0,3.469451e-17,1.336219,...,0.635976,1.562332,1.706007,1.482590,4.217859,4.988212,3.975046,3.076662,3.587075,2.930244
min,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,3.174270e-02,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.845182e+06,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.0,3.174270e-02,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.689508e+06,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.0,3.174270e-02,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.525968e+06,2.000000,1.000000,0.000000,2.000000,0.000000,1.0,0.0,3.174270e-02,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.361032e+06,92.000000,193.000000,84.000000,77.000000,1.000000,1.0,0.0,3.174270e-02,37.000000,...,11.000000,16.000000,16.000000,16.000000,117.000000,117.000000,117.000000,119.000000,119.000000,119.000000


In [31]:
user_df.head(20)

,User_id,User_receive_count,User_consume_count,User_used_count,User_not_used_count,User_used_coupon_rate,User_used_coupon_rate_max,User_used_coupon_rate_min,User_used_coupon_rate_mean,User_receive_coupon_merchant_count,...,User_distance_used_min,User_duration_used_mean,User_duration_used_max,User_duration_used_min,User_previous_duration_used_mean,User_previous_duration_used_max,User_previous_duration_used_min,User_next_duration_used_mean,User_next_duration_used_max,User_next_duration_used_min
0,4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35,4.0,0.0,0.0,4.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.031743,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,110,3.0,0.0,0.0,3.0,0.0,1.0,0.0,0.031743,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,144,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,147,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,165,2.0,14.0,0.0,2.0,0.0,1.0,0.0,0.031743,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,173,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,184,2.0,1.0,0.0,2.0,0.0,1.0,0.0,0.031743,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
user_df.to_csv('lcm_train_user_features.csv', index=False, header=True)

### 商户特征的提取

在模型的训练中，不再使用User_id，因为User_id不带有任何信息，我们需要从各种信息中获取信息来刻画一个User_id

+ 商家优惠券被领取次数
+ 商家优惠券被领取后不核销次数
+ 商家优惠券被领取后核销次数
+ 商家优惠券被领取后核销率
+ 商家优惠券核销的平均/最小/最大消费折率
+ 核销商家优惠券的不同用户数量，及其占领取不同的用户比重
+ 商家优惠券平均每个用户核销多少张
+ 商家被核销过的不同优惠券数量
+ 商家被核销过的不同优惠券数量占所有领取过的不同优惠券数量的比重
+ 商家平均每种优惠券核销多少张
+ 商家被核销优惠券中的平均/最小/最大用户-商家距离

以下不参与

+ 商家被核销优惠券的平均时间率

In [32]:
# 获取所有在线下消费过的商户id
merchant_df = df['Merchant_id'].drop_duplicates()
merchant_df = merchant_df.to_frame()

In [33]:
merchant_receive_count = received_df.groupby(['Merchant_id']).size()
merchant_consume_count = consume_df.groupby(['Merchant_id']).size()
merchant_used_count = used_coupon_df.groupby(['Merchant_id']).size()

# 商户特征
merchant_df = join(merchant_df, 'Merchant_receive_count', merchant_receive_count, 'Merchant_id')
merchant_df = join(merchant_df, 'Merchant_consume_count', merchant_consume_count, 'Merchant_id')
merchant_df = join(merchant_df, 'Merchant_used_count', merchant_used_count, 'Merchant_id')
merchant_df = merchant_df.fillna(0)

merchant_df['Merchant_not_used_count'] = merchant_df.apply(lambda r: r['Merchant_receive_count'] - r['Merchant_used_count'], axis=1)

# 商户15天内线下门店领取优惠券后进行核销率
merchant_df['Merchant_used_coupon_rate'] = merchant_df.apply(lambda r: r['Merchant_used_count'] / r['Merchant_receive_count'], axis=1)

merchant_df = merchant_df.fillna(0)
merchant_df = merchant_df.replace(math.inf, 0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [34]:
merchant_df['Merchant_used_coupon_rate_max'] = merchant_df['Merchant_used_coupon_rate'].max()
merchant_df['Merchant_used_coupon_rate_min'] = merchant_df['Merchant_used_coupon_rate'].min()
merchant_df['Merchant_used_coupon_rate_mean'] = merchant_df['Merchant_used_coupon_rate'].mean()

merchant_receive_coupon_user_count =  received_df[['User_id','Merchant_id']].drop_duplicates().groupby(['Merchant_id']).size()
merchant_consume_user_count =  consume_df[['User_id','Merchant_id']].drop_duplicates().groupby(['Merchant_id']).size()
merchant_used_coupon_user_count =  used_coupon_df[['User_id','Merchant_id']].drop_duplicates().groupby(['Merchant_id']).size()

merchant_df = join(merchant_df, 'Merchant_receive_coupon_user_count', merchant_receive_coupon_user_count, 'Merchant_id')
merchant_df = join(merchant_df, 'Merchant_consume_user_count', merchant_consume_user_count, 'Merchant_id')
merchant_df = join(merchant_df, 'Merchant_used_coupon_user_count', merchant_used_coupon_user_count, 'Merchant_id')
merchant_df = merchant_df.fillna(0)

user_count = df['User_id'].drop_duplicates().count()

merchant_df['Merchant_receive_coupon_user_occ'] = merchant_df['Merchant_receive_coupon_user_count'] / user_count
merchant_df['Merchant_consume_user_occ'] = merchant_df['Merchant_consume_user_count'] / user_count
merchant_df['Merchant_used_coupon_user_occ'] = merchant_df['Merchant_used_coupon_user_count'] / user_count

merchant_receive_different_coupon_count =  received_df[['Merchant_id','Coupon_id']].drop_duplicates().groupby(['Merchant_id']).size()
merchant_used_different_coupon_count =  used_coupon_df[['Merchant_id','Coupon_id']].drop_duplicates().groupby(['Merchant_id']).size()

merchant_df = join(merchant_df, 'Merchant_receive_different_coupon_count', merchant_receive_different_coupon_count, 'Merchant_id')
merchant_df = join(merchant_df, 'Merchant_used_different_coupon_count', merchant_used_different_coupon_count, 'Merchant_id')
merchant_df = merchant_df.fillna(0)

merchant_df['Merchant_receive_different_coupon_occ'] = merchant_df['Merchant_receive_different_coupon_count'] / coupon_count
merchant_df['Merchant_used_different_coupon_occ'] = merchant_df['Merchant_used_different_coupon_count'] / coupon_count

merchant_df['Merchant_receive_coupon_mean'] = merchant_df['Merchant_receive_count'] / merchant_df['Merchant_receive_coupon_user_count']
merchant_df['Merchant_used_coupon_mean'] = merchant_df['Merchant_used_count'] / merchant_df['Merchant_used_coupon_user_count']

merchant_df = merchant_df.replace(math.inf, 0)
merchant_df = merchant_df.fillna(0)

In [35]:
merchant_df['Merchant_receive_different_coupon_avg'] = merchant_df['Merchant_receive_count'] / merchant_df['Merchant_receive_different_coupon_count']
merchant_df['Merchant_used_different_coupon_avg'] = merchant_df['Merchant_receive_count'] / merchant_df['Merchant_used_coupon_user_count']

merchant_df = merchant_df.replace(math.inf, 0)
merchant_df = merchant_df.fillna(0)

In [36]:
merchant_distance_used_mean = used_coupon_df[['Merchant_id', 'Distance']].groupby(['Merchant_id']).mean()
merchant_distance_used_max = used_coupon_df[['Merchant_id', 'Distance']].groupby(['Merchant_id']).max()
merchant_distance_used_min = used_coupon_df[['Merchant_id', 'Distance']].groupby(['Merchant_id']).min()

merchant_distance_df = pd.DataFrame({'Merchant_id': merchant_distance_used_mean.index, 'Merchant_distance_used_mean':merchant_distance_used_mean['Distance'], 'Merchant_distance_used_max':merchant_distance_used_max['Distance'], 'Merchant_distance_used_min':merchant_distance_used_min['Distance']})

merchant_df = pd.merge(merchant_df, merchant_distance_df, on=['Merchant_id'], how='left')
merchant_df = merchant_df.fillna(0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: FutureWarning: 'Merchant_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
merchant_duration_used_mean = used_coupon_df[['Merchant_id', 'Duration']].groupby(['Merchant_id']).mean()
merchant_duration_used_max = used_coupon_df[['Merchant_id', 'Duration']].groupby(['Merchant_id']).max()
merchant_duration_used_min = used_coupon_df[['Merchant_id', 'Duration']].groupby(['Merchant_id']).min()

merchant_duration_df = pd.DataFrame({'Merchant_id': merchant_duration_used_mean.index, 'Merchant_duration_used_mean':merchant_duration_used_mean['Duration'], 'Merchant_duration_used_max':merchant_duration_used_max['Duration'], 'Merchant_duration_used_min':merchant_duration_used_min['Duration']})

merchant_df = pd.merge(merchant_df, merchant_duration_df, on=['Merchant_id'], how='left')
merchant_df = merchant_df.fillna(0)

In [38]:
merchant_previous_duration_used_mean = used_coupon_df[['Merchant_id', 'Previous_duration']].groupby(['Merchant_id']).mean()
merchant_previous_duration_used_max = used_coupon_df[['Merchant_id', 'Previous_duration']].groupby(['Merchant_id']).max()
merchant_previous_duration_used_min = used_coupon_df[['Merchant_id', 'Previous_duration']].groupby(['Merchant_id']).min()

merchant_previous_duration_df = pd.DataFrame({'Merchant_id': merchant_previous_duration_used_mean.index, 'Merchant_previous_duration_used_mean':merchant_previous_duration_used_mean['Previous_duration'], 'Merchant_previous_duration_used_max':merchant_previous_duration_used_max['Previous_duration'], 'Merchant_previous_duration_used_min':merchant_previous_duration_used_min['Previous_duration']})

merchant_df = pd.merge(merchant_df, merchant_previous_duration_df, on=['Merchant_id'], how='left')
merchant_df = merchant_df.fillna(0)

In [39]:
merchant_next_duration_used_mean = used_coupon_df[['Merchant_id', 'Next_duration']].groupby(['Merchant_id']).mean()
merchant_next_duration_used_max = used_coupon_df[['Merchant_id', 'Next_duration']].groupby(['Merchant_id']).max()
merchant_next_duration_used_min = used_coupon_df[['Merchant_id', 'Next_duration']].groupby(['Merchant_id']).min()

merchant_next_duration_df = pd.DataFrame({'Merchant_id': merchant_next_duration_used_mean.index, 'Merchant_next_duration_used_mean':merchant_next_duration_used_mean['Next_duration'], 'Merchant_next_duration_used_max':merchant_next_duration_used_max['Next_duration'], 'Merchant_next_duration_used_min':merchant_next_duration_used_min['Next_duration']})

merchant_df = pd.merge(merchant_df, merchant_next_duration_df, on=['Merchant_id'], how='left')
merchant_df = merchant_df.fillna(0)

In [40]:
merchant_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8313 entries, 0 to 8312
Data columns (total 35 columns):
Merchant_id                                8313 non-null int64
Merchant_receive_count                     8313 non-null float64
Merchant_consume_count                     8313 non-null float64
Merchant_used_count                        8313 non-null float64
Merchant_not_used_count                    8313 non-null float64
Merchant_used_coupon_rate                  8313 non-null float64
Merchant_used_coupon_rate_max              8313 non-null float64
Merchant_used_coupon_rate_min              8313 non-null float64
Merchant_used_coupon_rate_mean             8313 non-null float64
Merchant_receive_coupon_user_count         8313 non-null float64
Merchant_consume_user_count                8313 non-null float64
Merchant_used_coupon_user_count            8313 non-null float64
Merchant_receive_coupon_user_occ           8313 non-null float64
Merchant_consume_user_occ                  8313 no

In [41]:
merchant_df.describe()

,Merchant_id,Merchant_receive_count,Merchant_consume_count,Merchant_used_count,Merchant_not_used_count,Merchant_used_coupon_rate,Merchant_used_coupon_rate_max,Merchant_used_coupon_rate_min,Merchant_used_coupon_rate_mean,Merchant_receive_coupon_user_count,...,Merchant_distance_used_min,Merchant_duration_used_mean,Merchant_duration_used_max,Merchant_duration_used_min,Merchant_previous_duration_used_mean,Merchant_previous_duration_used_max,Merchant_previous_duration_used_min,Merchant_next_duration_used_mean,Merchant_next_duration_used_max,Merchant_next_duration_used_min
count,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.0,8313.0,8.313000e+03,8313.000000,...,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000
mean,4422.263924,89.855527,89.563094,4.367136,85.488392,0.057188,1.0,0.0,5.718828e-02,76.236136,...,0.396728,1.305360,1.827740,0.888007,2.278780,5.408998,1.102129,0.980365,2.759293,0.414892
std,2557.309228,1717.521887,818.631699,67.828342,1681.845843,0.156095,0.0,0.0,1.387862e-17,1499.546588,...,1.295472,3.021323,4.208722,2.451521,9.231488,17.728908,7.897459,4.377897,11.325247,3.512936
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,5.718828e-02,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2205.000000,0.000000,6.000000,0.000000,0.000000,0.000000,1.0,0.0,5.718828e-02,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4421.000000,0.000000,14.000000,0.000000,0.000000,0.000000,1.0,0.0,5.718828e-02,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6638.000000,4.000000,36.000000,0.000000,3.000000,0.000000,1.0,0.0,5.718828e-02,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8856.000000,122781.000000,36238.000000,4216.000000,121233.000000,1.000000,1.0,0.0,5.718828e-02,106932.000000,...,11.000000,16.000000,16.000000,16.000000,110.000000,117.000000,110.000000,98.000000,119.000000,98.000000


In [42]:
merchant_df.to_csv('lcm_train_merchant_features.csv', index=False, header=True)

### 优惠券特征的提取

+ 优惠券类型(直接优惠为0, 满减为1)
+ 优惠券折率
+ 满减优惠券的最低消费
+ 历史出现次数
+ 历史核销次数
+ 历史核销率
+ 历史核销时间率
+ 领取优惠券是一周的第几天
+ 领取优惠券是一月的第几天
+ 历史上用户领取该优惠券次数
+ 历史上用户消费该优惠券次数
+ 历史上用户对该优惠券的核销率

In [43]:
# 获取所有在线下消费过的优惠券id
coupon_df = df[df['Coupon_id']>0]['Coupon_id'].drop_duplicates()
coupon_df = coupon_df.to_frame()

In [44]:
coupon_received_count = received_df.groupby(['Coupon_id']).size()
coupon_used_count = used_coupon_df.groupby(['Coupon_id']).size()

coupon_df = join(coupon_df, 'Coupon_received_count', coupon_received_count, 'Coupon_id')
coupon_df = join(coupon_df, 'Coupon_used_count', coupon_used_count, 'Coupon_id')
coupon_df = coupon_df.fillna(0)

coupon_df['Coupon_used_rate'] = coupon_df['Coupon_used_count'] / coupon_df['Coupon_received_count']

coupon_df = coupon_df.replace(math.inf, 0)
coupon_df = coupon_df.fillna(0)

In [45]:
coupon_duration_used_mean = used_coupon_df[['Coupon_id', 'Duration']].groupby(['Coupon_id']).mean()
coupon_duration_used_max = used_coupon_df[['Coupon_id', 'Duration']].groupby(['Coupon_id']).max()
coupon_duration_used_min = used_coupon_df[['Coupon_id', 'Duration']].groupby(['Coupon_id']).min()

coupon_duration_df = pd.DataFrame({'Coupon_id': coupon_duration_used_mean.index, 'Coupon_duration_used_mean':coupon_duration_used_mean['Duration'], 'Coupon_duration_used_max':coupon_duration_used_max['Duration'], 'Coupon_duration_used_min':coupon_duration_used_min['Duration']})

coupon_df = pd.merge(coupon_df, coupon_duration_df, on=['Coupon_id'], how='left')
coupon_df = coupon_df.fillna(0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: FutureWarning: 'Coupon_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
coupon_distance_used_mean = used_coupon_df[['Coupon_id', 'Distance']].groupby(['Coupon_id']).mean()
coupon_distance_used_max = used_coupon_df[['Coupon_id', 'Distance']].groupby(['Coupon_id']).max()
coupon_distance_used_min = used_coupon_df[['Coupon_id', 'Distance']].groupby(['Coupon_id']).min()

coupon_distance_df = pd.DataFrame({'Coupon_id': coupon_distance_used_mean.index, 'Coupon_distance_used_mean':coupon_distance_used_mean['Distance'], 'Coupon_distance_used_max':coupon_distance_used_max['Distance'], 'Coupon_distance_used_min':coupon_distance_used_min['Distance']})

coupon_df = pd.merge(coupon_df, coupon_distance_df, on=['Coupon_id'], how='left')
coupon_df = coupon_df.fillna(0)

In [47]:
coupon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4483 entries, 0 to 4482
Data columns (total 10 columns):
Coupon_id                    4483 non-null int64
Coupon_received_count        4483 non-null int64
Coupon_used_count            4483 non-null float64
Coupon_used_rate             4483 non-null float64
Coupon_duration_used_mean    4483 non-null float64
Coupon_duration_used_max     4483 non-null float64
Coupon_duration_used_min     4483 non-null float64
Coupon_distance_used_mean    4483 non-null float64
Coupon_distance_used_max     4483 non-null float64
Coupon_distance_used_min     4483 non-null float64
dtypes: float64(8), int64(2)
memory usage: 385.3 KB


In [48]:
coupon_df.describe()

,Coupon_id,Coupon_received_count,Coupon_used_count,Coupon_used_rate,Coupon_duration_used_mean,Coupon_duration_used_max,Coupon_duration_used_min,Coupon_distance_used_mean,Coupon_distance_used_max,Coupon_distance_used_min
count,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000
mean,7082.062681,166.622574,8.098149,0.151606,3.436500,4.597145,2.447468,1.325063,2.102833,1.014499
std,4051.735886,1446.092303,70.866063,0.224066,4.131739,5.492976,3.612363,2.086032,3.334190,1.837682
min,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3568.500000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7204.000000,7.000000,1.000000,0.055556,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,10582.500000,20.000000,2.000000,0.222222,6.333333,9.000000,3.000000,1.444444,2.000000,1.000000
max,14045.000000,46676.000000,2434.000000,1.000000,16.000000,16.000000,16.000000,11.000000,11.000000,11.000000


In [49]:
coupon_df.to_csv('lcm_train_coupon_features.csv', index=False, header=True)

### 用户-商家特征提取

+ 用户领取商家的优惠券次数
+ 用户领取商家的优惠券后不核销次数
+ 用户领取商家的优惠券后核销次数
+ 用户领取商家的优惠券后核销率
+ 用户对每个商家的不核销次数占用户总的不核销次数的比重
+ 用户对每个商家的优惠券核销次数占用户总的核销次数的比重
+ 用户对每个商家的不核销次数占商家总的不核销次数的比重
+ 用户对每个商家的优惠券核销次数占商家总的核销次数的比重

In [50]:
# 获取所有在线下消费过的商户id
user_merchant_df = df[['User_id','Merchant_id']].drop_duplicates()

In [51]:
user_merchant_receive_count = received_df.groupby(['User_id', 'Merchant_id']).size()
user_merchant_consume_count = consume_df.groupby(['User_id', 'Merchant_id']).size()
user_merchant_used_count = used_coupon_df.groupby(['User_id', 'Merchant_id']).size()

user_merchant_df = multi_join(user_merchant_df, 'User_merchant_receive_count', user_merchant_receive_count, ['Merchant_id', 'User_id'])
user_merchant_df = multi_join(user_merchant_df, 'User_merchant_consume_count', user_merchant_consume_count, ['Merchant_id', 'User_id'])
user_merchant_df = multi_join(user_merchant_df, 'User_merchant_used_count', user_merchant_used_count, ['Merchant_id', 'User_id'])
user_merchant_df = user_merchant_df.fillna(0)

user_merchant_df['User_merchant_not_used_count'] = user_merchant_df.apply(lambda r: r['User_merchant_receive_count'] - r['User_merchant_used_count'], axis=1)

# 用户领取优惠券次数
user_receive_count = received_df.groupby(['User_id']).size()
# 用户15天内线下门店消费并核销优惠券次数
user_used_count = used_coupon_df.groupby(['User_id']).size()

user_merchant_df = join(user_merchant_df, 'User_receive_count', user_receive_count, 'User_id')
user_merchant_df = join(user_merchant_df, 'User_used_count', user_used_count, 'User_id')

user_merchant_df['User_merchant_used_coupon_rate'] = user_merchant_df['User_merchant_used_count'] / user_merchant_df['User_used_count']
user_merchant_df['User_merchant_not_used_coupon_rate'] = user_merchant_df['User_merchant_not_used_count'] / user_merchant_df['User_used_count']

user_merchant_df = user_merchant_df.fillna(0)
user_merchant_df = user_merchant_df.replace(math.inf, 0)

In [52]:
merchant_receive_count = received_df.groupby(['Merchant_id']).size()
merchant_used_count = used_coupon_df.groupby(['Merchant_id']).size()

user_merchant_df = join(user_merchant_df, 'Merchant_receive_count', merchant_receive_count, 'Merchant_id')
user_merchant_df = join(user_merchant_df, 'Merchant_used_count', merchant_used_count, 'Merchant_id')

user_merchant_df['User_merchant_used_coupon_rate_4_merchant'] = user_merchant_df['User_merchant_used_count'] / user_merchant_df['Merchant_used_count']
user_merchant_df['User_merchant_not_used_coupon_rate_4_merchant'] = user_merchant_df['User_merchant_not_used_count'] / user_merchant_df['Merchant_used_count']

user_merchant_df = user_merchant_df.fillna(0)
user_merchant_df = user_merchant_df.replace(math.inf, 0)

In [53]:
user_merchant_duration_used_mean = used_coupon_df[['User_id', 'Merchant_id', 'Duration']].groupby(['User_id', 'Merchant_id']).mean()
user_merchant_duration_used_max = used_coupon_df[['User_id', 'Merchant_id', 'Duration']].groupby(['User_id', 'Merchant_id']).max()
user_merchant_duration_used_min = used_coupon_df[['User_id', 'Merchant_id',  'Duration']].groupby(['User_id', 'Merchant_id']).min()

user_merchant_duration_used_mean = user_merchant_duration_used_mean.rename(columns={'Duration':'User_merchant_duration_used_mean'})
user_merchant_df = pd.merge(user_merchant_df, user_merchant_duration_used_mean, on=['Merchant_id', 'User_id'], how='left')

user_merchant_duration_used_max = user_merchant_duration_used_max.rename(columns={'Duration':'User_merchant_duration_used_max'})
user_merchant_df = pd.merge(user_merchant_df, user_merchant_duration_used_max, on=['Merchant_id', 'User_id'], how='left')

user_merchant_duration_used_min = user_merchant_duration_used_min.rename(columns={'Duration':'User_merchant_duration_used_min'})
user_merchant_df = pd.merge(user_merchant_df, user_merchant_duration_used_min, on=['Merchant_id', 'User_id'], how='left')

user_merchant_df = user_merchant_df.fillna(0)

In [54]:
user_merchant_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788994 entries, 0 to 788993
Data columns (total 17 columns):
User_id                                          788994 non-null int64
Merchant_id                                      788994 non-null int64
User_merchant_receive_count                      788994 non-null float64
User_merchant_consume_count                      788994 non-null float64
User_merchant_used_count                         788994 non-null float64
User_merchant_not_used_count                     788994 non-null float64
User_receive_count                               788994 non-null float64
User_used_count                                  788994 non-null float64
User_merchant_used_coupon_rate                   788994 non-null float64
User_merchant_not_used_coupon_rate               788994 non-null float64
Merchant_receive_count                           788994 non-null float64
Merchant_used_count                              788994 non-null float64
User_merchant_use

In [55]:
user_merchant_df.describe()

,User_id,Merchant_id,User_merchant_receive_count,User_merchant_consume_count,User_merchant_used_count,User_merchant_not_used_count,User_receive_count,User_used_count,User_merchant_used_coupon_rate,User_merchant_not_used_coupon_rate,Merchant_receive_count,Merchant_used_count,User_merchant_used_coupon_rate_4_merchant,User_merchant_not_used_coupon_rate_4_merchant,User_merchant_duration_used_mean,User_merchant_duration_used_max,User_merchant_duration_used_min
count,7.889940e+05,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000,788994.000000
mean,3.687331e+06,4053.113619,0.946736,0.943655,0.046013,0.900723,2.721055,0.120382,0.030645,0.046997,27686.364785,532.518294,0.002297,0.033146,0.195408,0.209915,0.182247
std,2.125528e+06,2392.515109,0.745636,2.830380,0.374041,0.655894,3.017459,0.674639,0.170186,0.274815,39715.484753,837.641990,0.038496,0.143059,1.265619,1.356518,1.214146
min,4.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.841280e+06,2099.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1847.000000,22.000000,0.000000,0.000474,0.000000,0.000000,0.000000
50%,3.692988e+06,3381.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,10533.000000,95.000000,0.000000,0.002358,0.000000,0.000000,0.000000
75%,5.531516e+06,6424.000000,1.000000,1.000000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,33427.000000,729.000000,0.000000,0.017241,0.000000,0.000000,0.000000
max,7.361032e+06,8856.000000,85.000000,126.000000,84.000000,64.000000,92.000000,84.000000,1.000000,20.000000,122781.000000,4216.000000,1.000000,18.000000,16.000000,16.000000,16.000000


In [56]:
user_merchant_df.to_csv('lcm_train_user_merchant_features.csv', index=False, header=True)

### 用户特征提取

#### 线上

+ 用户线上购买率
+ 用户线上领取率
+ 用户线上不消费次数
+ 用户线上优惠券核销次数
+ 用户线上优惠券核销率
+ 用户线下不消费次数占线上线下总的不消费次数的比重
+ 用户线下的优惠券核销次数占线上线下总的优惠券核销次数的比重
+ 用户线下领取的记录数量占总的记录数量的比重

不参与

+ 用户线上操作次数
+ 用户线上点击率

In [57]:
def check_is_in_day_consume(row):
    
    if row['Coupon_id'] == 'fixed':
        return 0
    
    if float(row['Coupon_id']) > 0 and float(row['Date_received']) > 0 and float(row['Date']) > 0:
        date_received = datetime.strptime(str(int(row['Date_received'])), '%Y%m%d')
        date_consumed = datetime.strptime(str(int(row['Date'])), '%Y%m%d')
        delta = date_consumed - date_received
        if delta.days < 16:
            return 1
        else:
            return 0
    
    return 0

online_df['Is_in_day_consume'] = online_df.apply(lambda row: check_is_in_day_consume(row), axis=1)

In [58]:
online_df['Coupon_id'] = online_df['Coupon_id'].replace('fixed', 0)
online_df['Coupon_id'] = online_df['Coupon_id'].astype('int64', copy=True)

# 领取优惠券的信息
online_received_df = online_df[online_df['Coupon_id'] > 0]

# 消费的信息
online_consume_df = online_df[online_df['Date'] > 0]

# 消费同时15天内使用优惠券的信息
online_used_coupon_df = online_df[online_df['Is_in_day_consume'] == 1]

# 获取所有在线下消费过的用户id
user_online_df = online_df['User_id'].drop_duplicates()
user_online_df = user_online_df.to_frame()

# 用户领取优惠券次数
online_user_receive_count = online_received_df.groupby(['User_id']).size()
# 用户线上门店消费次数
online_user_consume_count = online_consume_df.groupby(['User_id']).size()
# 用户15天内线下门店消费并核销优惠券次数
online_user_used_count = online_used_coupon_df.groupby(['User_id']).size()

# 用户特征
user_online_df = join(user_online_df, 'Online_user_receive_count', online_user_receive_count, 'User_id')
user_online_df = join(user_online_df, 'Online_user_consume_count', online_user_consume_count, 'User_id')
user_online_df = join(user_online_df, 'Online_user_used_count', online_user_used_count, 'User_id')
user_online_df = user_online_df.fillna(0)
user_online_df['Online_user_not_used_count'] = user_online_df.apply(lambda r: r['Online_user_receive_count'] - r['Online_user_used_count'], axis=1)

# 用户15天内线下门店领取优惠券后进行核销率
user_online_df['Online_user_used_coupon_rate'] = user_online_df.apply(lambda r: r['Online_user_used_count'] / r['Online_user_receive_count'], axis=1)
user_online_df = user_online_df.fillna(0)
user_online_df = user_online_df.replace(math.inf, 0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


In [59]:
user_online_df = join(user_online_df, 'User_receive_count', user_receive_count, 'User_id')
user_online_df = join(user_online_df, 'User_consume_count', user_consume_count, 'User_id')
user_online_df = join(user_online_df, 'User_used_count', user_used_count, 'User_id')
user_online_df['User_not_used_count'] = user_online_df.apply(lambda r: r['User_receive_count'] - r['User_used_count'], axis=1)
user_online_df = user_online_df.fillna(0)

In [60]:
user_online_df['User_offline_consume_rate'] = user_online_df.apply(lambda r: r['User_consume_count'] / (r['Online_user_consume_count'] + r['User_consume_count']), axis=1)  
user_online_df['User_offline_used_rate'] = user_online_df.apply(lambda r: r['User_used_count'] / (r['Online_user_used_count'] + r['User_used_count']), axis=1)
user_online_df['User_offline_no_consume_coupon_rate'] = user_online_df.apply(lambda r: r['User_not_used_count'] / (r['Online_user_not_used_count'] + r['User_not_used_count']), axis=1) 

user_online_df = user_online_df.fillna(0)
user_online_df = user_online_df.replace(math.inf, 0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.
/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
user_online_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762858 entries, 0 to 762857
Data columns (total 13 columns):
User_id                                762858 non-null int64
Online_user_receive_count              762858 non-null float64
Online_user_consume_count              762858 non-null float64
Online_user_used_count                 762858 non-null float64
Online_user_not_used_count             762858 non-null float64
Online_user_used_coupon_rate           762858 non-null float64
User_receive_count                     762858 non-null float64
User_consume_count                     762858 non-null float64
User_used_count                        762858 non-null float64
User_not_used_count                    762858 non-null float64
User_offline_consume_rate              762858 non-null float64
User_offline_used_rate                 762858 non-null float64
User_offline_no_consume_coupon_rate    762858 non-null float64
dtypes: float64(12), int64(1)
memory usage: 81.5 MB


In [62]:
user_online_df.describe()

,User_id,Online_user_receive_count,Online_user_consume_count,Online_user_used_count,Online_user_not_used_count,Online_user_used_coupon_rate,User_receive_count,User_consume_count,User_used_count,User_not_used_count,User_offline_consume_rate,User_offline_used_rate,User_offline_no_consume_coupon_rate
count,7.628580e+05,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000,762858.000000
mean,9.280525e+06,0.971099,14.123111,0.106743,0.864356,0.042533,0.505930,0.523380,0.025260,0.034959,0.066912,0.016374,0.008814
std,4.765184e+06,2.713198,34.512140,0.449378,2.529456,0.173094,1.275924,2.416383,0.265088,0.456168,0.194914,0.125287,0.088948
min,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.808160e+06,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.104637e+07,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.327289e+07,1.000000,15.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.550000e+07,240.000000,5786.000000,69.000000,240.000000,1.000000,92.000000,193.000000,70.000000,68.000000,1.000000,1.000000,1.000000


In [63]:
user_online_df.to_csv('lcm_train_user_online_features.csv', index=False, header=True)

## 训练数据做处理

In [64]:
# user_df = pd.read_csv('lcm_train_user_features.csv')
df = pd.merge(df, user_df, on=['User_id'], how='left')

In [65]:
# merchant_df = pd.read_csv('lcm_train_merchant_features.csv')
df = pd.merge(df, merchant_df, on=['Merchant_id'], how='left')

In [66]:
# coupon_df = pd.read_csv('lcm_train_coupon_features.csv')
df = pd.merge(df, coupon_df, on=['Coupon_id'], how='left')

In [67]:
# user_merchant_df = pd.read_csv('lcm_train_user_merchant_features.csv')
user_merchant_df = user_merchant_df.drop(['User_receive_count', 'User_used_count', 'Merchant_receive_count', 'Merchant_used_count'], axis=1)
user_merchant_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788994 entries, 0 to 788993
Data columns (total 13 columns):
User_id                                          788994 non-null int64
Merchant_id                                      788994 non-null int64
User_merchant_receive_count                      788994 non-null float64
User_merchant_consume_count                      788994 non-null float64
User_merchant_used_count                         788994 non-null float64
User_merchant_not_used_count                     788994 non-null float64
User_merchant_used_coupon_rate                   788994 non-null float64
User_merchant_not_used_coupon_rate               788994 non-null float64
User_merchant_used_coupon_rate_4_merchant        788994 non-null float64
User_merchant_not_used_coupon_rate_4_merchant    788994 non-null float64
User_merchant_duration_used_mean                 788994 non-null float64
User_merchant_duration_used_max                  788994 non-null float64
User_merchant_dur

In [68]:
df = pd.merge(df, user_merchant_df, on=['User_id', 'Merchant_id'], how='left')

In [69]:
# user_online_df = pd.read_csv('lcm_train_user_online_features.csv')
user_online_df = user_online_df.drop(['User_receive_count', 'User_consume_count', 'User_used_count', 'User_not_used_count'], axis=1)
user_online_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762858 entries, 0 to 762857
Data columns (total 9 columns):
User_id                                762858 non-null int64
Online_user_receive_count              762858 non-null float64
Online_user_consume_count              762858 non-null float64
Online_user_used_count                 762858 non-null float64
Online_user_not_used_count             762858 non-null float64
Online_user_used_coupon_rate           762858 non-null float64
User_offline_consume_rate              762858 non-null float64
User_offline_used_rate                 762858 non-null float64
User_offline_no_consume_coupon_rate    762858 non-null float64
dtypes: float64(8), int64(1)
memory usage: 58.2 MB


In [70]:
df = pd.merge(df, user_online_df, on=['User_id'], how='left')

In [71]:
df = df.fillna(0)

In [72]:
user_distance_receive_count = received_df.groupby(['User_id', 'Distance']).size()
user_distance_consume_count = consume_df.groupby(['User_id', 'Distance']).size()
user_distance_used_count = used_coupon_df.groupby(['User_id', 'Distance']).size()

# 用户-距离特征
df = multi_join(df, 'User_distance_receive_count', user_distance_receive_count, ['User_id', 'Distance'])
df = multi_join(df, 'User_distance_consume_count', user_distance_consume_count, ['User_id', 'Distance'])
df = multi_join(df, 'User_distance_used_count', user_distance_used_count, ['User_id', 'Distance'])
df = df.fillna(0)

df['User_distance_receive_rate'] = df['User_distance_receive_count'] / df['User_receive_count']
df['User_distance_consume_rate'] = df['User_distance_consume_count'] / df['User_consume_count']
df['User_distance_used_rate'] = df['User_distance_used_count'] / df['User_receive_count']
df = df.fillna(0)

In [73]:
user_coupon_type_receive_count = received_df.groupby(['User_id', 'Coupon_type']).size()
user_coupon_type_used_count = used_coupon_df.groupby(['User_id', 'Coupon_type']).size()

# 用户-优惠券类型特征
df = multi_join(df, 'User_coupon_type_receive_count', user_coupon_type_receive_count, ['User_id', 'Coupon_type'])
df = multi_join(df, 'User_coupon_type_used_count', user_coupon_type_used_count, ['User_id', 'Coupon_type'])
df = df.fillna(0)

df['User_coupon_type_receive_rate'] = df['User_coupon_type_receive_count'] / df['User_receive_count']
df['User_coupon_type_used_rate'] = df['User_coupon_type_used_count'] / df['User_receive_count']

In [74]:
user_coupon_receive_count = received_df.groupby(['User_id', 'Coupon_id']).size()
user_coupon_used_count = used_coupon_df.groupby(['User_id', 'Coupon_id']).size()

# 用户-优惠券特征
df = multi_join(df, 'User_coupon_receive_count', user_coupon_receive_count, ['User_id', 'Coupon_id'])
df = multi_join(df, 'User_coupon_used_count', user_coupon_used_count, ['User_id', 'Coupon_id'])
df = df.fillna(0)

df['User_coupon_receive_rate'] = df['User_coupon_receive_count'] / df['User_receive_count']
df['User_coupon_used_rate'] = df['User_coupon_used_count'] / df['User_receive_count']
df = df.fillna(0)

In [75]:
merchant_distance_receive_count = received_df.groupby(['Merchant_id', 'Distance']).size()
merchant_distance_consume_count = consume_df.groupby(['Merchant_id', 'Distance']).size()
merchant_distance_used_count = used_coupon_df.groupby(['Merchant_id', 'Distance']).size()

# 商户-距离特征
df = multi_join(df, 'Merchant_distance_receive_count', merchant_distance_receive_count, ['Distance', 'Merchant_id'])
df = multi_join(df, 'Merchant_distance_consume_count', merchant_distance_consume_count, ['Distance', 'Merchant_id'])
df = multi_join(df, 'Merchant_distance_used_count', merchant_distance_used_count, ['Distance', 'Merchant_id'])
df = df.fillna(0)

df['Merchant_distance_receive_rate'] = df['Merchant_distance_receive_count'] / df['Merchant_receive_count']
df['Merchant_distance_used_rate'] = df['Merchant_distance_used_count'] / df['Merchant_receive_count']
df = df.fillna(0)

In [76]:
user_coupon_duration_used_mean = used_coupon_df[['User_id', 'Coupon_id', 'Duration']].groupby(['User_id', 'Coupon_id']).mean()
user_coupon_duration_used_max = used_coupon_df[['User_id', 'Coupon_id', 'Duration']].groupby(['User_id', 'Coupon_id']).max()
user_coupon_duration_used_min = used_coupon_df[['User_id', 'Coupon_id', 'Duration']].groupby(['User_id', 'Coupon_id']).min()

user_coupon_duration_used_mean = user_coupon_duration_used_mean.rename(columns={'Duration':'User_coupon_duration_used_mean'})
df = pd.merge(df, user_coupon_duration_used_mean, on=['Coupon_id', 'User_id'], how='left')

user_coupon_duration_used_max = user_coupon_duration_used_max.rename(columns={'Duration':'User_coupon_duration_used_max'})
df = pd.merge(df, user_coupon_duration_used_max, on=['Coupon_id', 'User_id'], how='left')

user_coupon_duration_used_min = user_coupon_duration_used_min.rename(columns={'Duration':'User_coupon_duration_used_min'})
df = pd.merge(df, user_coupon_duration_used_min, on=['Coupon_id', 'User_id'], how='left')

In [77]:
user_received_date_count = df[['User_id', 'Date_received']].groupby(['User_id']).size()
df = multi_join(df, 'User_received_date_count', user_received_date_count, ['User_id'])

df = df.replace(math.inf, 0)
df = df.fillna(0)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448571 entries, 0 to 1448570
Columns: 133 entries, User_id to User_received_date_count
dtypes: float64(123), int64(9), object(1)
memory usage: 1.4+ GB


In [79]:
df.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Previous_date_received,Next_date_received,Duration,Previous_duration,...,User_coupon_used_rate,Merchant_distance_receive_count,Merchant_distance_consume_count,Merchant_distance_used_count,Merchant_distance_receive_rate,Merchant_distance_used_rate,User_coupon_duration_used_mean,User_coupon_duration_used_max,User_coupon_duration_used_min,User_received_date_count
count,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,...,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06,1.448571e+06
mean,3.690789e+06,4.123009e+03,3.656079e+03,3.324601e+00,1.039580e+07,1.036206e+07,1.039579e+07,1.039579e+07,2.635798e-01,2.384647e+00,...,1.619791e-02,5.522301e+03,4.088671e+03,3.291852e+02,3.477256e-01,4.523622e-02,1.783589e-01,2.058498e-01,1.567462e-01,9.694536e+00
std,2.123726e+06,2.393802e+03,4.634087e+03,3.504746e+00,1.007517e+07,1.007626e+07,1.007517e+07,1.007517e+07,2.080123e+00,1.069058e+01,...,1.085810e-01,1.000979e+04,8.076895e+03,8.248596e+02,3.040569e-01,8.696503e-02,1.195908e+00,1.372134e+00,1.109308e+00,1.320657e+01
min,4.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,1.845660e+06,2.099000e+03,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,7.000000e+01,5.200000e+01,1.000000e+00,8.029437e-02,1.714342e-04,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
50%,3.697100e+06,3.532000e+03,1.110000e+02,1.000000e+00,2.016012e+07,2.016011e+07,2.016012e+07,2.016012e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,1.262000e+03,3.700000e+02,1.600000e+01,2.497214e-01,3.846154e-03,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00
75%,5.531531e+06,6.424000e+03,7.610000e+03,4.000000e+00,2.016020e+07,2.016042e+07,2.016020e+07,2.016020e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,6.163000e+03,3.780000e+03,2.330000e+02,5.944584e-01,5.886303e-02,0.000000e+00,0.000000e+00,0.000000e+00,1.200000e+01
max,7.361032e+06,8.856000e+03,1.404500e+04,1.100000e+01,2.016043e+07,2.016063e+07,2.016043e+07,2.016043e+07,9.700000e+01,1.210000e+02,...,1.000000e+00,4.248400e+04,3.456800e+04,3.972000e+03,1.000000e+00,1.000000e+00,1.600000e+01,1.600000e+01,1.600000e+01,2.600000e+02


In [80]:
df.to_csv('lcm_train_features.csv', index=False, header=True)

### 训练测试数据处理

In [81]:
df_train_test = pd.read_csv('ccf_offline_stage1_train.csv')
df_train_test = df_train_test[df_train_test['Date_received']>=20160501]
df_train_test = df_train_test.fillna(0)
df_train_test = base_data_process(df_train_test)

In [82]:
df_train_test = pd.merge(df_train_test, user_df, on=['User_id'], how='left')

In [83]:
df_train_test = pd.merge(df_train_test, merchant_df, on=['Merchant_id'], how='left')

In [84]:
df_train_test = pd.merge(df_train_test, coupon_df, on=['Coupon_id'], how='left')

In [85]:
df_train_test = pd.merge(df_train_test, user_merchant_df, on=['User_id', 'Merchant_id'], how='left')

In [86]:
df_train_test = pd.merge(df_train_test, user_online_df, on=['User_id'], how='left')

In [87]:
# 用户-距离特征
df_train_test = multi_join(df_train_test, 'User_distance_receive_count', user_distance_receive_count, ['User_id', 'Distance'])
df_train_test = multi_join(df_train_test, 'User_distance_consume_count', user_distance_consume_count, ['User_id', 'Distance'])
df_train_test = multi_join(df_train_test, 'User_distance_used_count', user_distance_used_count, ['User_id', 'Distance'])
df_train_test = df_train_test.fillna(0)

df_train_test['User_distance_receive_rate'] = df_train_test['User_distance_receive_count'] / df_train_test['User_receive_count']
df_train_test['User_distance_consume_rate'] = df_train_test['User_distance_consume_count'] / df_train_test['User_consume_count']
df_train_test['User_distance_used_rate'] = df_train_test['User_distance_used_count'] / df_train_test['User_receive_count']
df_train_test = df_train_test.fillna(0)

In [88]:
# 用户-优惠券类型特征
df_train_test = multi_join(df_train_test, 'User_coupon_type_receive_count', user_coupon_type_receive_count, ['User_id', 'Coupon_type'])
df_train_test = multi_join(df_train_test, 'User_coupon_type_used_count', user_coupon_type_used_count, ['User_id', 'Coupon_type'])
df_train_test = df_train_test.fillna(0)

df_train_test['User_coupon_type_receive_rate'] = df_train_test['User_coupon_type_receive_count'] / df_train_test['User_receive_count']
df_train_test['User_coupon_type_used_rate'] = df_train_test['User_coupon_type_used_count'] / df_train_test['User_receive_count']

In [89]:
# 用户-优惠券特征
df_train_test = multi_join(df_train_test, 'User_coupon_receive_count', user_coupon_receive_count, ['User_id', 'Coupon_id'])
df_train_test = multi_join(df_train_test, 'User_coupon_used_count', user_coupon_used_count, ['User_id', 'Coupon_id'])
df_train_test = df_train_test.fillna(0)

df_train_test['User_coupon_receive_rate'] = df_train_test['User_coupon_receive_count'] / df_train_test['User_receive_count']
df_train_test['User_coupon_used_rate'] = df_train_test['User_coupon_used_count'] / df_train_test['User_receive_count']
df_train_test = df_train_test.fillna(0)

In [90]:
# 商户-距离特征
df_train_test = multi_join(df_train_test, 'Merchant_distance_receive_count', merchant_distance_receive_count, ['Distance', 'Merchant_id'])
df_train_test = multi_join(df_train_test, 'Merchant_distance_consume_count', merchant_distance_consume_count, ['Distance', 'Merchant_id'])
df_train_test = multi_join(df_train_test, 'Merchant_distance_used_count', merchant_distance_used_count, ['Distance', 'Merchant_id'])
df_train_test = df_train_test.fillna(0)

df_train_test['Merchant_distance_receive_rate'] = df_train_test['Merchant_distance_receive_count'] / df_train_test['Merchant_receive_count']
df_train_test['Merchant_distance_used_rate'] = df_train_test['Merchant_distance_used_count'] / df_train_test['Merchant_receive_count']
df_train_test = df_train_test.fillna(0)

In [91]:
user_coupon_duration_used_mean = user_coupon_duration_used_mean.rename(columns={'Duration':'User_coupon_duration_used_mean'})
df_train_test = pd.merge(df_train_test, user_coupon_duration_used_mean, on=['Coupon_id', 'User_id'], how='left')

user_coupon_duration_used_max = user_coupon_duration_used_max.rename(columns={'Duration':'User_coupon_duration_used_max'})
df_train_test = pd.merge(df_train_test, user_coupon_duration_used_max, on=['Coupon_id', 'User_id'], how='left')

user_coupon_duration_used_min = user_coupon_duration_used_min.rename(columns={'Duration':'User_coupon_duration_used_min'})
df_train_test = pd.merge(df_train_test, user_coupon_duration_used_min, on=['Coupon_id', 'User_id'], how='left')

In [92]:
user_received_date_count = df_train_test[['User_id', 'Date_received']].groupby(['User_id']).size()
df_train_test = multi_join(df_train_test, 'User_received_date_count', user_received_date_count, ['User_id'])

df_train_test = df_train_test.replace(math.inf, 0)
df_train_test = df_train_test.fillna(0)

In [93]:
df_train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306313 entries, 0 to 306312
Columns: 133 entries, User_id to User_received_date_count
dtypes: float64(122), int64(10), object(1)
memory usage: 313.2+ MB


In [94]:
df_train_test.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Previous_date_received,Next_date_received,Duration,Previous_duration,...,User_coupon_used_rate,Merchant_distance_receive_count,Merchant_distance_consume_count,Merchant_distance_used_count,Merchant_distance_receive_rate,Merchant_distance_used_rate,User_coupon_duration_used_mean,User_coupon_duration_used_max,User_coupon_duration_used_min,User_received_date_count
count,3.063130e+05,306313.000000,306313.000000,306313.000000,3.063130e+05,3.063130e+05,3.063130e+05,3.063130e+05,306313.000000,306313.000000,...,306313.000000,306313.000000,306313.000000,306313.000000,306313.000000,306313.000000,306313.000000,306313.000000,306313.000000,306313.000000
mean,3.681999e+06,3640.616419,6145.500612,2.719506,2.016055e+07,2.135494e+06,2.016048e+07,2.016048e+07,0.923350,2.955421,...,0.004549,1480.663191,2290.509299,97.605361,0.310735,0.042851,0.051792,0.060428,0.043873,2.836572
std,2.122008e+06,2589.367153,4135.385856,2.960127,4.063352e+01,6.204236e+06,3.642670e+04,3.642670e+04,3.544654,6.206802,...,0.057551,2458.038976,3939.460026,232.369510,0.293922,0.079737,0.670206,0.768143,0.605317,4.015028
min,4.000000e+00,2.000000,1.000000,1.000000,2.016050e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.842402e+06,1080.000000,2418.000000,1.000000,2.016052e+07,0.000000e+00,2.016052e+07,2.016052e+07,0.000000,0.000000,...,0.000000,7.000000,49.000000,0.000000,0.049887,0.000000,0.000000,0.000000,0.000000,1.000000
50%,3.684421e+06,3585.000000,4958.000000,1.000000,2.016052e+07,0.000000e+00,2.016052e+07,2.016052e+07,0.000000,0.000000,...,0.000000,234.000000,369.000000,10.000000,0.179685,0.006390,0.000000,0.000000,0.000000,2.000000
75%,5.518531e+06,5800.000000,9746.000000,3.000000,2.016060e+07,0.000000e+00,2.016060e+07,2.016060e+07,0.000000,2.000000,...,0.000000,1571.000000,2470.000000,87.000000,0.530172,0.044261,0.000000,0.000000,0.000000,3.000000
max,7.360961e+06,8856.000000,14045.000000,11.000000,2.016062e+07,2.016063e+07,2.016062e+07,2.016062e+07,55.000000,46.000000,...,1.000000,42484.000000,20099.000000,2037.000000,1.000000,1.000000,16.000000,16.000000,16.000000,90.000000


In [95]:
df_train_test.to_csv('lcm_train_test_features.csv', index=False, header=True)

## 预测数据做处理

In [106]:
test_df = pd.read_csv('ccf_offline_stage1_test_revised.csv')

In [107]:
test_df = test_df.fillna(0)
def base_predict_data_process(df):
    df = df.sort_values(by=['User_id', 'Date_received'], ascending=True)

    df['Previous_user_id'] = df['User_id'].shift(1)
    df['Previous_date_received'] = df['Date_received'].shift(1)

    df['Next_user_id'] = df['User_id'].shift(-1)
    df['Next_date_received'] = df['Date_received'].shift(-1)

    df.fillna(0)
    
    df['Distance'] = df['Distance'] + 1
    df['Previous_duration'] = df.apply(lambda row: cal_previous_duration(row), axis=1)
    df['Next_duration'] = df.apply(lambda row: cal_next_duration(row), axis=1)
    
    df = df.drop(['Next_user_id', 'Previous_user_id'], axis=1)
    
    df['Base_consume'] = df.apply(lambda row: base_consume(row), axis=1)
    df['Day_in_month_received'] = df.apply(lambda row: get_day_in_month_4_received_day(row), axis=1)
    df['Day_in_week_received'] = df.apply(lambda row: get_day_in_week_4_received_day(row), axis=1)
    df['Discount'] = df.apply(lambda row: cal_discount(row), axis=1)
    df['Coupon_type'] = df.apply(lambda row: set_coupon_type(row), axis=1)
    
    return df

test_df = base_predict_data_process(test_df)
test_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Previous_date_received,Next_date_received,Previous_duration,Next_duration,Base_consume,Day_in_month_received,Day_in_week_received,Discount,Coupon_type
23866,209,5032,7557,20:5,2.0,20160721,NaN,20160721.0,0,1,20.0,0.0,0.0,0.750000,1
23867,209,5032,825,20:5,2.0,20160721,20160721.0,20160703.0,1,0,20.0,0.0,0.0,0.750000,1
23793,215,599,5488,20:1,1.0,20160703,20160721.0,20160721.0,0,0,20.0,0.0,0.0,0.950000,1
61826,316,2436,3992,30:5,1.0,20160721,20160703.0,20160712.0,0,0,30.0,0.0,0.0,0.833333,1
23851,417,3507,12465,50:1,1.0,20160712,20160721.0,20160706.0,0,0,50.0,0.0,0.0,0.980000,1


In [108]:
test_df = pd.merge(test_df, user_df, on=['User_id'], how='left')

In [109]:
test_df = pd.merge(test_df, merchant_df, on=['Merchant_id'], how='left')

In [110]:
test_df = pd.merge(test_df, coupon_df, on=['Coupon_id'], how='left')

In [111]:
test_df = pd.merge(test_df, user_merchant_df, on=['User_id', 'Merchant_id'], how='left')

In [112]:
test_df = pd.merge(test_df, user_online_df, on=['User_id'], how='left')

In [113]:
# 用户-距离特征
test_df = multi_join(test_df, 'User_distance_receive_count', user_distance_receive_count, ['User_id', 'Distance'])
test_df = multi_join(test_df, 'User_distance_consume_count', user_distance_consume_count, ['User_id', 'Distance'])
test_df = multi_join(test_df, 'User_distance_used_count', user_distance_used_count, ['User_id', 'Distance'])
test_df = test_df.fillna(0)

test_df['User_distance_receive_rate'] = test_df['User_distance_receive_count'] / test_df['User_receive_count']
test_df['User_distance_consume_rate'] = test_df['User_distance_consume_count'] / test_df['User_consume_count']
test_df['User_distance_used_rate'] = test_df['User_distance_used_count'] / test_df['User_receive_count']
test_df = test_df.fillna(0)

In [115]:
# 用户-优惠券类型特征
test_df = multi_join(test_df, 'User_coupon_type_receive_count', user_coupon_type_receive_count, ['User_id', 'Coupon_type'])
test_df = multi_join(test_df, 'User_coupon_type_used_count', user_coupon_type_used_count, ['User_id', 'Coupon_type'])
test_df = test_df.fillna(0)

test_df['User_coupon_type_receive_rate'] = test_df['User_coupon_type_receive_count'] / test_df['User_receive_count']
test_df['User_coupon_type_used_rate'] = test_df['User_coupon_type_used_count'] / test_df['User_receive_count']

In [114]:
# 用户-优惠券特征
test_df = multi_join(test_df, 'User_coupon_receive_count', user_coupon_receive_count, ['User_id', 'Coupon_id'])
test_df = multi_join(test_df, 'User_coupon_used_count', user_coupon_used_count, ['User_id', 'Coupon_id'])
test_df = test_df.fillna(0)

test_df['User_coupon_receive_rate'] = test_df['User_coupon_receive_count'] / test_df['User_receive_count']
test_df['User_coupon_used_rate'] = test_df['User_coupon_used_count'] / test_df['User_receive_count']
test_df = test_df.fillna(0)

In [116]:
# 商户-距离特征
test_df = multi_join(test_df, 'Merchant_distance_receive_count', merchant_distance_receive_count, ['Distance', 'Merchant_id'])
test_df = multi_join(test_df, 'Merchant_distance_consume_count', merchant_distance_consume_count, ['Distance', 'Merchant_id'])
test_df = multi_join(test_df, 'Merchant_distance_used_count', merchant_distance_used_count, ['Distance', 'Merchant_id'])
test_df = test_df.fillna(0)

test_df['Merchant_distance_receive_rate'] = test_df['Merchant_distance_receive_count'] / test_df['Merchant_receive_count']
test_df['Merchant_distance_used_rate'] = test_df['Merchant_distance_used_count'] / test_df['Merchant_receive_count']
test_df = test_df.fillna(0)

In [117]:
user_coupon_duration_used_mean = user_coupon_duration_used_mean.rename(columns={'Duration':'User_coupon_duration_used_mean'})
test_df = pd.merge(test_df, user_coupon_duration_used_mean, on=['Coupon_id', 'User_id'], how='left')

user_coupon_duration_used_max = user_coupon_duration_used_max.rename(columns={'Duration':'User_coupon_duration_used_max'})
test_df = pd.merge(test_df, user_coupon_duration_used_max, on=['Coupon_id', 'User_id'], how='left')

user_coupon_duration_used_min = user_coupon_duration_used_min.rename(columns={'Duration':'User_coupon_duration_used_min'})
test_df = pd.merge(test_df, user_coupon_duration_used_min, on=['Coupon_id', 'User_id'], how='left')

In [118]:
user_received_date_count = test_df[['User_id', 'Date_received']].groupby(['User_id']).size()
test_df = multi_join(test_df, 'User_received_date_count', user_received_date_count, ['User_id'])

test_df = test_df.replace(math.inf, 0)
test_df = test_df.fillna(0)

In [119]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113640 entries, 0 to 113639
Columns: 130 entries, User_id to User_received_date_count
dtypes: float64(121), int64(8), object(1)
memory usage: 113.6+ MB


In [120]:
test_df.to_csv('lcm_test_features.csv', index=False, header=True)

## 进一步迭代方向

In [ ]:
# 暂时没有添加等结果来看

user_receive_date_count = df.groupby(['User_id','Date_received']).size()

indexer =  user_receive_date_count.index.to_frame()
user_receive_date_df = pd.DataFrame({'User_id': indexer['User_id'], 'Date_received':indexer['Date_received'], 'User_receive_date_count':user_receive_date_count.values})

df = pd.merge(df, user_receive_date_df[['User_receive_date_count']], on=['User_id', 'Date_received'], how='left')
df = df.fillna(0)
df.head(5)